$$
  H = \sum_n \left[ \frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2+\frac{a}{2}V(\phi_n)^2 + aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \right. \nonumber \\
             \quad \left.+(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right) + \frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \right], 
$$

# Sparse matrix form

In [1]:
import scipy
from scipy.sparse import eye, kron, coo_matrix
from scipy.sparse.linalg import eigsh

import numpy as np
from susy_qm import create_matrix
from functools import reduce

In [22]:
# Parameters
N = 2
cutoff = 16
a = 1.0
potential = "linear"
#potential = "quadratic"
c = -0.2

In [23]:
def kron_tensor(size, site, total_sites, operator):

    I = eye(size, format='coo', dtype=np.complex128)
    operators = [I] * total_sites
    operators[site] = operator
  
    return reduce(kron, operators) 

In [24]:
I_b = eye(cutoff ** N, format='coo')
I_f = eye(2 ** N, format='coo')
dim = I_b.size * I_f.size

zero_qop = coo_matrix((I_b.size,I_b.size), dtype=np.complex128)
zero_cop = coo_matrix((I_f.size,I_f.size), dtype=np.complex128)

H_b = coo_matrix((I_b.size,I_b.size), dtype=np.complex128)
H_f = coo_matrix((I_f.size,I_f.size), dtype=np.complex128)
H_bi = coo_matrix((dim, dim), dtype=np.complex128)

H = coo_matrix((dim, dim), dtype=np.complex128)

q = coo_matrix(create_matrix(cutoff, 'q'))
p = coo_matrix(create_matrix(cutoff, 'p'))
chi = coo_matrix([[0, 1], [0, 0]], dtype=np.complex128)
chidag = coo_matrix([[0, 0], [1, 0]], dtype=np.complex128)

for n in range(N):

    print(n)
    q_n = kron_tensor(cutoff, n, N, q)
    p_n = kron_tensor(cutoff, n, N, p)

    chi_n = kron_tensor(2, n, N, chi)
    chidag_n = kron_tensor(2, n, N, chidag)

    #Boson terms
    # Kinetic term
    p2 = coo_matrix(p_n @ p_n / (2 * a))  

    # Potential term
    if potential == "linear":
        W_prime = q_n  # W'(q) = q
        W_double_prime = I_b  # W''(q) = 1
    elif potential == "quadratic":
        W_prime = c * I_b + coo_matrix(q_n @ q_n)  # W'(q) = c + q^2
        W_double_prime = 2*q_n  # W''(q) = 2q

    potential_term = (a / 2) * coo_matrix(W_prime @ W_prime)

    boundary_conditions = 'dirichlet'
    #boundary_conditions = 'periodic'

    if boundary_conditions == 'dirichlet':
        if n == 0:
            q_nm1 = zero_qop
            q_np1 = kron_tensor(cutoff, (n + 1), N, q)
            chi_np1 = kron_tensor(2, (n + 1), N, chi)
            chidag_np1 = kron_tensor(2, (n + 1) , N, chidag)

        elif n == N-1:
            q_nm1 = kron_tensor(cutoff, (n - 1), N, q)
            q_np1 = zero_qop
            chi_np1 = zero_cop
            chidag_np1 = zero_cop

        else:
            q_np1 = kron_tensor(cutoff, (n + 1), N, q)
            chi_np1 = kron_tensor(2, (n + 1), N, chi)
            chidag_np1 = kron_tensor(2, (n + 1) , N, chidag)
            q_nm1 = kron_tensor(cutoff, (n - 1), N, q)

    elif boundary_conditions == 'periodic':
        
        q_np1 = kron_tensor(cutoff, (n + 1) % N, N, q)
        q_nm1 = kron_tensor(cutoff, (n - 1) % N, N, q)

        if n == N-1:
            chi_np1 = kron_tensor(2, 0, N, -1*chi)
            chidag_np1 = kron_tensor(2, 0, N, -1*chidag)

        else:
            chi_np1 = kron_tensor(2, (n + 1), N, chi)
            chidag_np1 = kron_tensor(2, (n + 1) , N, chidag)
  
        

    gradient = coo_matrix((q_np1 - q_nm1) / (2 * a))
    gradient_term = (a / 2) * (gradient @ gradient)

    potential_gradient_term = a * (W_prime @ gradient)

    H_b += (p2 + potential_term + gradient_term + potential_gradient_term)

    # Boson-Fermion term
    commutator_term = kron(W_double_prime, ((-1) ** n) * coo_matrix(chidag_n @ chi_n - 0.5 * I_f), format='coo')
    H_bi += commutator_term

    #Fermion term
    fermion_hopping = 0.5*(chidag_n @ chi_np1 + chidag_np1 @ chi_n)
    H_f += (fermion_hopping)

H = kron(H_b, I_f, format='coo') + kron(I_b, H_f, format='coo') + H_bi
   

0
1


In [25]:
# Compute the k smallest eigenvalues of the sparse Hamiltonian
num_eigenvalues = 6
#eigenvalues, eigenvectors = eigsh(H, k=num_eigenvalues, which='SA')
eigenvalues = np.sort(eigsh(H, k=num_eigenvalues, which='SA')[0])
print("Eigenvalues:", np.sort(eigenvalues))

Eigenvalues: [-7.20495969e-16  1.11803399e+00  1.11803399e+00  1.11803399e+00
  1.11803399e+00  2.23606798e+00]


In [96]:
from qiskit.quantum_info import SparsePauliOp

In [97]:
SparsePauliOp.from_operator(H.todense())

SparsePauliOp(['IIIIIIIIIIII', 'IIIIIIIIIIXX', 'IIIIIIIIIIYY', 'IIIIIIIIIXIX', 'IIIIIIIIIXXI', 'IIIIIIIIIYIY', 'IIIIIIIIIYYI', 'IIIIIIIIXIIZ', 'IIIIIIIXIIII', 'IIIIIIIXXIIZ', 'IIIIIIIXZIII', 'IIIIIIIYYIIZ', 'IIIIIIIZIIII', 'IIIIIIIZXIIZ', 'IIIIIIIZZIII', 'IIIIIIXIIIII', 'IIIIIIXIZIII', 'IIIIIIXXIIII', 'IIIIIIXXXIIZ', 'IIIIIIXXZIII', 'IIIIIIXYYIIZ', 'IIIIIIXZIIII', 'IIIIIIXZZIII', 'IIIIIIYXYIIZ', 'IIIIIIYYIIII', 'IIIIIIYYXIIZ', 'IIIIIIYYZIII', 'IIIIIIZIIIII', 'IIIIIIZIXIIZ', 'IIIIIIZIZIII', 'IIIIIIZXIIII', 'IIIIIIZXXIIZ', 'IIIIIIZXZIII', 'IIIIIIZYYIIZ', 'IIIIIIZZIIII', 'IIIIIIZZXIIZ', 'IIIIIIZZZIII', 'IIIIIXIIIIZI', 'IIIIIXIIXIII', 'IIIIIXIXIIII', 'IIIIIXIXXIII', 'IIIIIXIXZIII', 'IIIIIXIYYIII', 'IIIIIXIZIIII', 'IIIIIXIZXIII', 'IIIIIXIZZIII', 'IIIIIXXXIIII', 'IIIIIXXXXIII', 'IIIIIXXXZIII', 'IIIIIXXYYIII', 'IIIIIXYXYIII', 'IIIIIXYYIIII', 'IIIIIXYYXIII', 'IIIIIXYYZIII', 'IIIIIXZIIIII', 'IIIIIXZIXIII', 'IIIIIXZIZIII', 'IIIIIXZXIIII', 'IIIIIXZXXIII', 'IIIIIXZXZIII', 'IIIIIXZYYIII', 'IIIIIXZZ

In [ ]:
# Initialize the Hamiltonian as a sparse matrix
I_b = eye(cutoff ** N, format='coo')
I_f = eye(2 ** N, format='coo')
dim = I_b.size * I_f.size
H = coo_matrix((dim, dim), dtype=np.complex128)

# Define operators as sparse matrices
q = coo_matrix(create_matrix(cutoff, 'q'))
p = coo_matrix(create_matrix(cutoff, 'p'))
chi = coo_matrix([[0, 1], [0, 0]], dtype=np.complex128)
chidag = coo_matrix([[0, 0], [1, 0]], dtype=np.complex128)

boundary_conditions = 'dirichlet' 
#boundary_conditions ='periodic'

for n in range(N):
    
    q_n = kron_tensor(cutoff, n, N, q)
    p_n = kron_tensor(cutoff, n, N, p)

    chi_n = kron_tensor(2, n, N, chi)
    chidag_n = kron_tensor(2, n, N, chidag)
    

    # Kinetic term
    p2 = coo_matrix(p_n @ p_n / (2 * a))  
    kinetic_term = kron(p2, I_f, format='coo') 

    # Potential term
    W_prime = q_n  # W'(q) = q
    W_double_prime = I_b  # W''(q) = 1
    potential = coo_matrix(W_prime @ W_prime * (a / 2))
    potential_term = kron(potential, I_f, format='coo')

    # Fermionic term
    commutator_term = ((-1) ** n) * kron(W_double_prime, coo_matrix(chidag_n @ chi_n - 0.5 * I_f), format='coo')

    H += kinetic_term + potential_term + commutator_term 
    
    
    # Cross-site terms
    if boundary_conditions == 'dirichlet':
        print('dirichlet')
        # Set operators to zero matrices at boundary
        zero_qop = coo_matrix((I_b.size,I_b.size), dtype=np.complex128)
        zero_cop = coo_matrix((I_f.size,I_f.size), dtype=np.complex128)

        # Dirichlet BC's
        if n < N-1:
            print(n)
            q_np1 = kron_tensor(cutoff, (n + 1), N, q)
            chi_np1 = kron_tensor(2, (n + 1), N, chi)
            chidag_np1 = kron_tensor(2, (n + 1) , N, chidag)
        else:
            print(n, 'Zero')
            q_np1 = zero_qop
            chi_np1 = zero_cop
            chidag_np1 = zero_cop

        if n == 0:
            q_nm1 = zero_qop
        else:
            q_nm1 = kron_tensor(cutoff, (n - 1), N, q)
    
    elif boundary_conditions == 'periodic':
        print('periodic')
        q_np1 = kron_tensor(cutoff, (n + 1) % N, N, q)
        q_nm1 = kron_tensor(cutoff, (n - 1) % N, N, q)
        chi_np1 = kron_tensor(2, (n + 1) % N, N, chi)
        chidag_np1 = kron_tensor(2, (n + 1) % N, N, chidag)

    gradient = coo_matrix((q_np1 - q_nm1) / (2 * a))
    gradient_term = kron((a / 2) * (gradient @ gradient), I_f, format='coo')
    potential_gradient_term = kron(a * (W_prime @ gradient), I_f, format='coo')

    # Fermionic hopping
    fermion = coo_matrix((chidag_n @ chi_np1 + chidag_np1 @ chi_n)) / (2 * a)
    fermion_term = kron(I_b, fermion, format='coo')

    H += potential_gradient_term + fermion_term


In [15]:
# Compute the k smallest eigenvalues of the sparse Hamiltonian
num_eigenvalues = 6
eigenvalues, eigenvectors = eigsh(H, k=num_eigenvalues, which='SA')

print("Eigenvalues:", np.sort(eigenvalues))

Eigenvalues: [-0.11803399  0.88196601  0.88196601  1.          1.          1.88196601]
